<a href="https://colab.research.google.com/github/mariadiosdado/deep-learning-challenge/blob/main/AlphabetSoupCharity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 4.6 MB/s eta 0:00:00


In [53]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
import keras_tuner as kt

In [50]:
#  Import and read the charity_data.csv
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [19]:
application_df.columns

Index(['EIN', 'NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
       'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT',
       'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [20]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [21]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [22]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [23]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = (list(application_counts[application_counts<500].index))
application_types_to_replace

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [24]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [25]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [26]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [27]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

In [28]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [31]:
application_df.head(6)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
5,T3,Independent,C1200,Preservation,Trust,1,0,N,5000,1


In [30]:
# Convert categorical data to numeric with `pd.get_dummies`
dummified_df = pd.get_dummies(application_df, dtype=int)
dummified_df.head(6)

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
5,1,5000,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [32]:
# Split our preprocessed data into our features and target arrays
X = dummified_df.drop('IS_SUCCESSFUL', axis=1)
y = dummified_df['IS_SUCCESSFUL']

In [34]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)

In [35]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Compile, Train, and Evaluate the Model

In [59]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
def create_model(hp):
    model = tf.keras.models.Sequential()
    activation = hp.Choice("activation", ["relu","tanh","sigmoid"])

    model.add(Dense(units = hp.Int("num_neurons", 1,8, step=3),
                   activation=activation,
                    input_dim = X_train.shape[1]
                   ))

    for x in range(hp.Int("num_layers", 1,6)):
        model.add(Dense(units = hp.Int("num_layers_" + str(x+2), 1 , 10, step = 2),
                       activation=activation
                       ))

    model.add(Dense(units=1, activation = "sigmoid"))

    model.compile(loss = "binary_crossentropy",
                  optimizer = "adam",
                  metrics = ["accuracy"])

    return model

In [54]:
tuner = kt.Hyperband(
                    create_model,
                    objective = "val_accuracy",
                    max_epochs=10,
                    hyperband_iterations=2,
                    overwrite = True
)

In [55]:
tuner.search(X_train_scaled, y_train, epochs = 5, validation_data = (X_test_scaled, y_test))

Trial 60 Complete [00h 00m 19s]
val_accuracy: 0.7231487035751343

Best val_accuracy So Far: 0.7287463545799255
Total elapsed time: 00h 08m 59s


In [56]:
# Find best model Hyperparameters
best_hp = tuner.get_best_hyperparameters(1)[0]
best_hp.values

{'activation': 'tanh',
 'num_neurons': 7,
 'num_layers': 4,
 'num_layers_2': 5,
 'num_layers_3': 7,
 'num_layers_4': 3,
 'num_layers_5': 7,
 'num_layers_6': 9,
 'num_layers_7': 5,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [63]:
# Evaluate the model using the test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5567 - accuracy: 0.7287 - 519ms/epoch - 2ms/step
Loss: 0.5567478537559509, Accuracy: 0.7287463545799255


In [64]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(Dense(units =80, activation = 'tanh', input_dim = X_train.shape[1]))

# Second hidden layer
nn.add(Dense(units = 65, activation = 'tanh'))

# Third hidden layer
nn.add(Dense(units = 50, activation = 'tanh'))

# Fourth hidden layer
nn.add(Dense(units = 30, activation = 'tanh'))

# Fifth hidden layer
nn.add(Dense(units = 10, activation = 'tanh'))

# Output layer
nn.add(Dense(units = 1, activation = 'tanh'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                3520      
                                                                 
 dense_7 (Dense)             (None, 65)                5265      
                                                                 
 dense_8 (Dense)             (None, 50)                3300      
                                                                 
 dense_9 (Dense)             (None, 30)                1530      
                                                                 
 dense_10 (Dense)            (None, 10)                310       
                                                                 
 dense_11 (Dense)            (None, 1)                 11        
                                                                 
Total params: 13936 (54.44 KB)
Trainable params: 13936

In [65]:
# Compile the model
nn.compile(loss = "binary_crossentropy",
                  optimizer = "adam",
                  metrics = ["accuracy"])

In [67]:
# Train the model
nn.fit(X_train, y_train, epochs = 10, shuffle=True, verbose = 2)

Epoch 1/10
804/804 - 3s - loss: 8.2474 - accuracy: 0.4653 - 3s/epoch - 4ms/step
Epoch 2/10
804/804 - 1s - loss: 8.2474 - accuracy: 0.4653 - 1s/epoch - 1ms/step
Epoch 3/10
804/804 - 1s - loss: 8.2474 - accuracy: 0.4653 - 1s/epoch - 1ms/step
Epoch 4/10
804/804 - 1s - loss: 8.2474 - accuracy: 0.4653 - 1s/epoch - 1ms/step
Epoch 5/10
804/804 - 1s - loss: 8.2474 - accuracy: 0.4653 - 1s/epoch - 1ms/step
Epoch 6/10
804/804 - 1s - loss: 8.2474 - accuracy: 0.4653 - 1s/epoch - 1ms/step
Epoch 7/10
804/804 - 1s - loss: 8.2474 - accuracy: 0.4653 - 1s/epoch - 1ms/step
Epoch 8/10
804/804 - 1s - loss: 8.2474 - accuracy: 0.4653 - 1s/epoch - 1ms/step
Epoch 9/10
804/804 - 1s - loss: 8.2474 - accuracy: 0.4653 - 1s/epoch - 2ms/step
Epoch 10/10
804/804 - 1s - loss: 8.2474 - accuracy: 0.4653 - 1s/epoch - 2ms/step


In [68]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 3.8864 - accuracy: 0.4911 - 349ms/epoch - 1ms/step
Loss: 3.88641619682312, Accuracy: 0.4910787045955658


In [70]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")